Create spark session with python 3.10.13

In [9]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/14 22:47:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/04/14 22:47:38 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


Imported CSV file

In [10]:
original_df = spark.read.format("csv").option("header", "true").load("/home/karthi/Documents/Spark/BDA_File.csv")

Ensure CSV imported correctly

In [11]:
original_df.show()

24/04/14 22:47:52 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------+----------------+-------+---------+---+------------+-------------------+---------+-------------------+--------------+----------------+--------------+---------+-----------+------------------------+-------------+----+--------+-------+---------+---------+------+------------------+--------------------+-------------------+-----------------------+--------------------+-----------------+
|    short_name|player_positions|overall|potential|age|club_team_id|          club_name|league_id|        league_name|nationality_id|nationality_name|preferred_foot|weak_foot|skill_moves|international_reputation|    work_rate|pace|shooting|passing|dribbling|defending|physic|goalkeeping_diving|goalkeeping_handling|goalkeeping_kicking|goalkeeping_positioning|goalkeeping_reflexes|goalkeeping_speed|
+--------------+----------------+-------+---------+---+------------+-------------------+---------+-------------------+--------------+----------------+--------------+---------+-----------+---------------

Check datatype of attributes

In [12]:
for dtype in original_df.dtypes:
  print(f'{dtype[0]}, {dtype[1]}')

short_name, string
player_positions, string
overall, string
potential, string
age, string
club_team_id, string
club_name, string
league_id, string
league_name, string
nationality_id, string
nationality_name, string
preferred_foot, string
weak_foot, string
skill_moves, string
international_reputation, string
work_rate, string
pace, string
shooting, string
passing, string
dribbling, string
defending, string
physic, string
goalkeeping_diving, string
goalkeeping_handling, string
goalkeeping_kicking, string
goalkeeping_positioning, string
goalkeeping_reflexes, string
goalkeeping_speed, string


In [13]:
from pyspark.sql.types import StringType, IntegerType

fifa_df = original_df

# List of columns to be converted to string type
string_cols = ["short_name", "player_positions", "club_name", "league_name", 
         "nationality_name", "preferred_foot", "work_rate"]

# Convert specified columns to string type
for col in string_cols:
  fifa_df = fifa_df.withColumn(col, fifa_df[col].cast(StringType()))

# Get all column names
all_cols = fifa_df.columns

# List of columns to be converted to integer type
int_cols = list(set(all_cols) - set(string_cols))

# Convert the rest of the columns to integer type
for col in int_cols:
  fifa_df = fifa_df.withColumn(col, fifa_df[col].cast(IntegerType()))

fifa_df.show()

+--------------+----------------+-------+---------+---+------------+-------------------+---------+-------------------+--------------+----------------+--------------+---------+-----------+------------------------+-------------+----+--------+-------+---------+---------+------+------------------+--------------------+-------------------+-----------------------+--------------------+-----------------+
|    short_name|player_positions|overall|potential|age|club_team_id|          club_name|league_id|        league_name|nationality_id|nationality_name|preferred_foot|weak_foot|skill_moves|international_reputation|    work_rate|pace|shooting|passing|dribbling|defending|physic|goalkeeping_diving|goalkeeping_handling|goalkeeping_kicking|goalkeeping_positioning|goalkeeping_reflexes|goalkeeping_speed|
+--------------+----------------+-------+---------+---+------------+-------------------+---------+-------------------+--------------+----------------+--------------+---------+-----------+---------------

In [14]:
for dtype in fifa_df.dtypes:
  print(f'{dtype[0]}, {dtype[1]}')

short_name, string
player_positions, string
overall, int
potential, int
age, int
club_team_id, int
club_name, string
league_id, int
league_name, string
nationality_id, int
nationality_name, string
preferred_foot, string
weak_foot, int
skill_moves, int
international_reputation, int
work_rate, string
pace, int
shooting, int
passing, int
dribbling, int
defending, int
physic, int
goalkeeping_diving, int
goalkeeping_handling, int
goalkeeping_kicking, int
goalkeeping_positioning, int
goalkeeping_reflexes, int
goalkeeping_speed, int


1. Player Vesatility

In [35]:
from pyspark.sql.functions import size, split, when

# Create a new DataFrame with 'short_name' and 'versatility'
versatility_df = fifa_df.select(
  "short_name",
  when(size(split(col("player_positions"), ",")) - 1 >= 2, "very versatile")
  .when(size(split(col("player_positions"), ",")) - 1 == 1, "versatile")
  .otherwise("not versatile")
  .alias("versatility")
)

In [36]:
versatility_df.show(30)

+---------------+--------------+
|     short_name|   versatility|
+---------------+--------------+
|     K. MbappÃ©|     versatile|
|     E. Haaland| not versatile|
|   K. De Bruyne|     versatile|
|       L. Messi|     versatile|
|     K. Benzema|     versatile|
| R. Lewandowski| not versatile|
|    T. Courtois| not versatile|
|        H. Kane| not versatile|
|       Vini Jr.| not versatile|
|        Alisson| not versatile|
|          Rodri|     versatile|
|    RÃºben Dias| not versatile|
|      Neymar Jr| not versatile|
|  M. ter Stegen| not versatile|
|       Casemiro| not versatile|
|    V. van Dijk| not versatile|
|       M. Salah| not versatile|
|    F. Valverde|     versatile|
|     V. Osimhen| not versatile|
|        Ederson| not versatile|
|     J. Kimmich|very versatile|
|   A. Griezmann|     versatile|
|       J. Oblak| not versatile|
|Bruno Fernandes|     versatile|
| Bernardo Silva|     versatile|
|   M. Ã˜degaard|     versatile|
|     F. de Jong|     versatile|
|  G. Donn

Exported CSV file

In [ ]:
versatility_df.write.csv("/home/karthi/Documents/Spark/file.csv", header=True)

2. Highly talented youngster

In [20]:
from pyspark.sql.functions import col

# Create a new DataFrame with 'short_name', 'potential', and 'overall' where 'potential' > 80
new_df = fifa_df.select("short_name", "potential", "overall").where(col("potential") > 80)

In [21]:
# new_df.show()

filtered_df = new_df.filter(col("potential") - col("overall") > 3).select("short_name").withColumnRenamed("short_name", "highly_talented_youngsters")
filtered_df.show()

+--------------------------+
|highly_talented_youngsters|
+--------------------------+
|                  Vini Jr.|
|               F. Valverde|
|                J. Musiala|
|          K. Kvaratskhelia|
|                     Pedri|
|              Rafael LeÃ£o|
|             J. Bellingham|
|                 R. Araujo|
|                   B. Saka|
|                   Rodrygo|
|                  F. Wirtz|
|             T. HernÃ¡ndez|
|                A. Bastoni|
|                  P. Foden|
|                J. KoundÃ©|
|            A. TchouamÃ©ni|
|               I. Bennacer|
|                 F. Tomori|
|                  M. Diaby|
|        Gabriel Martinelli|
+--------------------------+
only showing top 20 rows



3. Ranking Players based on Ability on all aspects of the game ( top 500 )    

In [22]:
from pyspark.sql.functions import col, avg

new1_df = fifa_df

# Calculate the average of the specified columns
new1_df = fifa_df.withColumn(
  "ability_score",
  (col("pace") + col("shooting") + col("dribbling") + col("passing") + col("defending") + col("physic")) / 6
)

# Create a new DataFrame with 'short_name' and 'ability_score'
new1_df = new1_df.select("short_name", "ability_score")

# Rank the records in descending order of 'ability_score'
ranked_df = new1_df.orderBy(col("ability_score").desc())


ranked_df.show()

+--------------------+-----------------+
|          short_name|    ability_score|
+--------------------+-----------------+
|         F. Valverde|83.33333333333333|
|         L. Goretzka|             81.5|
|           S. Tonali|             81.0|
|     Marcos Llorente|80.83333333333333|
|        K. De Bruyne|80.66666666666667|
|       T. HernÃ¡ndez|             80.5|
|              E. Can|80.33333333333333|
|          N. Barella|80.16666666666667|
|           A. Rabiot|             80.0|
|           M. AcuÃ±a|             80.0|
|           A. Hakimi|             80.0|
|            R. James|             80.0|
|          F. de Jong|79.83333333333333|
|       J. Bellingham|79.83333333333333|
|        Mikel Merino|79.83333333333333|
|          R. De Paul|79.66666666666667|
|          J. Kimmich|             79.5|
|S. MilinkoviÄ‡-Sa...|             79.5|
|     Bruno Fernandes|79.33333333333333|
|        A. Griezmann|79.33333333333333|
+--------------------+-----------------+
only showing top

4. Average overall on each age range

In [23]:
from pyspark.sql.functions import avg

# Filter the DataFrame based on the age range and calculate the average 'overall' score
average_overall_young = fifa_df.filter((col("age") >= 16) & (col("age") <= 24)).agg(avg("overall")).first()[0]
average_overall_mid = fifa_df.filter((col("age") >= 25) & (col("age") <= 32)).agg(avg("overall")).first()[0]
average_overall_old = fifa_df.filter((col("age") >= 33) & (col("age") <= 39)).agg(avg("overall")).first()[0]

print(f'Average overall score of young players: {average_overall_young}')
print(f'Average overall score of mid-aged players: {average_overall_mid}')
print(f'Average overall score of old players: {average_overall_old}')

Average overall score of young players: 62.94553672316384
Average overall score of mid-aged players: 68.39448051948052
Average overall score of old players: 69.02387448840382


In [24]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
  .appName("Football Analytics") \
  .config("spark.driver.memory", "4g") \
  .getOrCreate()

24/04/14 22:49:03 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [25]:
from pyspark.sql.functions import col

# List of club names
clubs = ["Paris Saint Germain", "Manchester City", "FC Barcelona", "Real Madrid", 
     "FC Bayern MÃ¼nchen", "AtlÃ©tico Madrid", "Arsenal", "Borussia Dortmund"]

# Create a new DataFrame with players from the specified clubs
league_df = fifa_df.filter(col("club_name").isin(clubs)).select("short_name", "club_name", "overall")

In [26]:
league_df.show()

+--------------+-------------------+-------+
|    short_name|          club_name|overall|
+--------------+-------------------+-------+
|    K. MbappÃ©|Paris Saint Germain|     91|
|    E. Haaland|    Manchester City|     91|
|  K. De Bruyne|    Manchester City|     91|
|R. Lewandowski|       FC Barcelona|     90|
|   T. Courtois|        Real Madrid|     90|
|       H. Kane| FC Bayern MÃ¼nchen|     90|
|      Vini Jr.|        Real Madrid|     89|
|         Rodri|    Manchester City|     89|
|   RÃºben Dias|    Manchester City|     89|
| M. ter Stegen|       FC Barcelona|     89|
|   F. Valverde|        Real Madrid|     88|
|       Ederson|    Manchester City|     88|
|    J. Kimmich| FC Bayern MÃ¼nchen|     88|
|  A. Griezmann|   AtlÃ©tico Madrid|     88|
|      J. Oblak|   AtlÃ©tico Madrid|     88|
|Bernardo Silva|    Manchester City|     88|
|  M. Ã˜degaard|            Arsenal|     87|
|    F. de Jong|       FC Barcelona|     87|
| G. Donnarumma|Paris Saint Germain|     87|
|      G. 

In [27]:
record_count = league_df.count()
print(record_count)

243


5. Top 10 Players of each clubs

In [28]:
from pyspark.sql.functions import col, desc

def get_top_players(club_name, suggested_name):
    # Filter the DataFrame based on the club name and select the top 10 players based on 'overall' score
    top_players_df = fifa_df.filter(col("club_name") == club_name).orderBy(desc("overall")).limit(10).select(
      "short_name").withColumnRenamed("short_name", suggested_name)
    return top_players_df

# clubs = ["Paris Saint Germain", "Manchester City", "FC Barcelona", "Real Madrid", 
#          "FC Bayern MÃ¼nchen", "AtlÃ©tico Madrid", "Arsenal", "Borussia Dortmund"]

paris_df = get_top_players("Paris Saint Germain", "Paris Saint Germain")
manchester_df = get_top_players("Manchester City", "Manchester City")
barcelona_df = get_top_players("FC Barcelona", "FC Barcelona")
madrid_df = get_top_players("Real Madrid", "Real Madrid")
bayern_df = get_top_players("FC Bayern MÃ¼nchen", "FC Bayern Munchen")
atletico_df = get_top_players("AtlÃ©tico Madrid", "Atletico Madrid")
arsenal_df = get_top_players("Arsenal", "Arsenal")
dortmund_df = get_top_players("Borussia Dortmund", "Borussia Dortmund")


In [29]:
from pyspark.sql.functions import monotonically_increasing_id

# Add a common column 'id' to each DataFrame
paris_df = paris_df.withColumn("rank", monotonically_increasing_id() + 1)
manchester_df = manchester_df.withColumn("rank", monotonically_increasing_id() + 1)
barcelona_df = barcelona_df.withColumn("rank", monotonically_increasing_id() + 1)
madrid_df = madrid_df.withColumn("rank", monotonically_increasing_id() + 1)
bayern_df = bayern_df.withColumn("rank", monotonically_increasing_id() + 1)
atletico_df = atletico_df.withColumn("rank", monotonically_increasing_id() + 1)
arsenal_df = arsenal_df.withColumn("rank", monotonically_increasing_id() + 1)
dortmund_df = dortmund_df.withColumn("rank", monotonically_increasing_id() + 1)

# Join the DataFrames on the 'id' column
joined_df = paris_df.join(manchester_df, "rank", "outer") \
  .join(barcelona_df, "rank", "outer") \
  .join(madrid_df, "rank", "outer") \
  .join(bayern_df, "rank", "outer") \
  .join(atletico_df, "rank", "outer") \
  .join(arsenal_df, "rank", "outer") \
  .join(dortmund_df, "rank", "outer")

# Show the DataFrame
joined_df.show()

+----+-------------------+---------------+--------------+--------------+-----------------+---------------+------------------+-----------------+
|rank|Paris Saint Germain|Manchester City|  FC Barcelona|   Real Madrid|FC Bayern Munchen|Atletico Madrid|           Arsenal|Borussia Dortmund|
+----+-------------------+---------------+--------------+--------------+-----------------+---------------+------------------+-----------------+
|   1|         K. MbappÃ©|     E. Haaland|R. Lewandowski|   T. Courtois|          H. Kane|   A. Griezmann|      M. Ã˜degaard|         G. Kobel|
|   2|      G. Donnarumma|   K. De Bruyne| M. ter Stegen|      Vini Jr.|       J. Kimmich|       J. Oblak|           B. Saka|         N. SÃ¼le|
|   3|         Marquinhos|          Rodri|    F. de Jong|   F. Valverde|         M. Neuer|           Koke|           D. Rice|        J. Brandt|
|   4|       O. DembÃ©lÃ©|    RÃºben Dias|         Pedri|    L. ModriÄ‡|       J. Musiala|       M. Depay|         T. Partey| N. Schlott

In [30]:

league_rank_df = fifa_df.select("short_name", "league_name", "overall").orderBy(desc("overall")).limit(200)
league_rank_df.show()
league_rank_df.count()

+--------------+-------------------+-------+
|    short_name|        league_name|overall|
+--------------+-------------------+-------+
|    K. MbappÃ©|            Ligue 1|     91|
|    E. Haaland|     Premier League|     91|
|  K. De Bruyne|     Premier League|     91|
|      L. Messi|Major League Soccer|     90|
|    K. Benzema|         Pro League|     90|
|R. Lewandowski|            La Liga|     90|
|   T. Courtois|            La Liga|     90|
|       H. Kane|         Bundesliga|     90|
|      Vini Jr.|            La Liga|     89|
|       Alisson|     Premier League|     89|
|         Rodri|     Premier League|     89|
|   RÃºben Dias|     Premier League|     89|
|     Neymar Jr|         Pro League|     89|
| M. ter Stegen|            La Liga|     89|
|      Casemiro|     Premier League|     89|
|   V. van Dijk|     Premier League|     89|
|      M. Salah|     Premier League|     89|
|   F. Valverde|            La Liga|     88|
|    V. Osimhen|            Serie A|     88|
|       Ed

200

6. League ranking with top 200 Players

In [34]:
from pyspark.sql.functions import col

# Create a new DataFrame with distinct league names and the number of players in each league
rleague_df = league_rank_df.groupBy("league_name").count().withColumnRenamed("count", "number_of_players").orderBy(desc("count"))

# Show the DataFrame
rleague_df.show()

+-------------------+-----------------+
|        league_name|number_of_players|
+-------------------+-----------------+
|     Premier League|               55|
|            La Liga|               45|
|            Serie A|               35|
|         Bundesliga|               32|
|         Pro League|               14|
|            Ligue 1|               12|
|Major League Soccer|                3|
|      Liga Portugal|                3|
|          Super Lig|                1|
+-------------------+-----------------+



In [37]:
from pyspark.sql.functions import split, col

# Create a new DataFrame with 'player_positions', 'short_name', and 'overall'
new_df = fifa_df.select(
  split(col("player_positions"), ",")[0].alias("player_positions"),
  "short_name",
  "overall"
)

new_df.show()

+----------------+--------------+-------+
|player_positions|    short_name|overall|
+----------------+--------------+-------+
|              ST|    K. MbappÃ©|     91|
|              ST|    E. Haaland|     91|
|              CM|  K. De Bruyne|     91|
|              CF|      L. Messi|     90|
|              CF|    K. Benzema|     90|
|              ST|R. Lewandowski|     90|
|              GK|   T. Courtois|     90|
|              ST|       H. Kane|     90|
|              LW|      Vini Jr.|     89|
|              GK|       Alisson|     89|
|             CDM|         Rodri|     89|
|              CB|   RÃºben Dias|     89|
|              LW|     Neymar Jr|     89|
|              GK| M. ter Stegen|     89|
|             CDM|      Casemiro|     89|
|              CB|   V. van Dijk|     89|
|              RW|      M. Salah|     89|
|              CM|   F. Valverde|     88|
|              ST|    V. Osimhen|     88|
|              GK|       Ederson|     88|
+----------------+--------------+-

7. Top 3 rated players in each position  

In [40]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, desc

# Define a Window partitioned by 'player_positions', ordered by 'overall' in descending order
window = Window.partitionBy("player_positions").orderBy(desc("overall"))

# Add a row number within each Window partition
new_df = new_df.withColumn("row_number", row_number().over(window))

# Filter to keep only the top 3 players for each position
top_players_df = new_df.filter(col("row_number") <= 3).select("player_positions", "short_name", "overall")

# Show the DataFrame
top_players_df.show()
top_players_df.count()

+----------------+---------------+-------+
|player_positions|     short_name|overall|
+----------------+---------------+-------+
|             CAM|Bruno Fernandes|     88|
|             CAM|   M. Ã˜degaard|     87|
|             CAM|     J. Musiala|     86|
|              CB|    RÃºben Dias|     89|
|              CB|    V. van Dijk|     89|
|              CB|     Marquinhos|     87|
|             CDM|          Rodri|     89|
|             CDM|       Casemiro|     89|
|             CDM|     J. Kimmich|     88|
|              CF|       L. Messi|     90|
|              CF|     K. Benzema|     90|
|              CF|      C. Nkunku|     86|
|              CM|   K. De Bruyne|     91|
|              CM|    F. Valverde|     88|
|              CM| Bernardo Silva|     88|
|              GK|    T. Courtois|     90|
|              GK|        Alisson|     89|
|              GK|  M. ter Stegen|     89|
|              LB|  JoÃ£o Cancelo|     86|
|              LB|   A. Robertson|     86|
+----------

45

In [42]:
top_players_df.write.csv("/home/karthi/Documents/Spark/top_player.csv", header=True)